In [ ]:
import requests
from random import randint
from bs4 import BeautifulSoup 
import csv 

USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Firefox/68.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36",
]

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}


all_books = []

In [ ]:
st = 30001
en = 40000

In [ ]:
def fetch_book_details(book_id):
    """Fetch book details from a specific book page on rokomari.com."""
    book_url = f"https://www.rokomari.com/book/{book_id}"

    headers = HEADERS
    headers["User-Agent"] = USER_AGENTS[randint(0, len(USER_AGENTS) - 1)]

  
    response = requests.get(book_url, headers=headers)
    print(f"\n🔹 Fetching: {book_url}")
    print(f"🔹 Response Status Code: {response.status_code}")

    if response.ok:
        soup = BeautifulSoup(response.text, 'html.parser')

        book_name_tag = soup.find("h1", class_="detailsBookContainer_bookName__pLCtW")
        book_name = book_name_tag.text.strip() if book_name_tag else "N/A"

        subtitle_tag = soup.find("h2", class_="detailsBookContainer_subTitle__nEIOj")
        subtitle = subtitle_tag.text.strip() if subtitle_tag else "N/A"

        author_tag = soup.find("p", class_="detailsBookContainer_authorName__ZP0vX")
        author_names = ", ".join([a.text.strip() for a in author_tag.find_all("a")]) if author_tag else "N/A"

        category_tag = soup.find("p", class_="detailsBookContainer_category___lQrb")
        category = category_tag.find("a").text.strip() if category_tag else "N/A"

        sell_price_tag = soup.find("span", class_="sell-price")
        sell_price = sell_price_tag.text.strip().replace("TK.", "").strip() if sell_price_tag else "N/A"

        book_details = {
            "Book ID": book_id,
            "Book Name": book_name,
            "Subtitle": subtitle,
            "Author": author_names,
            "Category": category,
            "Price": sell_price
        }

        if book_name != "N/A":
            all_books.append(book_details)
            print(f"✅ Extracted: {book_name}")
    else:
        print(f"❌ Failed to fetch: {response.reason}")

In [ ]:
for book_id in range(st, en + 1):
    fetch_book_details(book_id)

In [ ]:
csv_filename = f"rokomari_books_{st}_to_{en}.csv"
with open(csv_filename, "w", newline="", encoding="utf-8") as f:
    if all_books:
        fieldnames = all_books[0].keys()  
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(all_books)
        print(f"✅ Data saved to {csv_filename}")
    else:
        print("❌ No book details found to save.")